Make connection with G-Drive



In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

#jenach dem müssen die dateipfade unten im code angepasst werden

Mounted at /content/gdrive


Packets installieren

In [ ]:
!pip install peewee pytorch-lightning
!wget https://storage.googleapis.com/chesspic/datasets/2021-07-31-lichess-evaluations-37MM.db.gz
!gzip -d "2021-07-31-lichess-evaluations-37MM.db.gz"
!rm "2021-07-31-lichess-evaluations-37MM.db.gz"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 56.8 MB/s eta 0:00:00
--2023-12-25 09:05:37--  https://storage.googleapis.com/chesspic/datasets/2021-07-31-lichess-evaluations-37MM.db.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.213.207, 173.194.215.207, 108.177.13.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.213.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1404305784 (1.3G) [application/x-gzip]
Saving to: ‘2021-07-31-lichess-evaluations-37MM.db.gz’

2021-07-31-lichess- 100%[===================>]   1.31G   160MB/s    in 10s     

2023-12-25 09:05:48 (129 MB/s) - ‘2021-07-31-lichess-evaluations-37MM.db.gz’ saved [1404305784/1404305784]

rm: cannot remove '2021-07-31-lichess-evaluations-37MM.db.gz': No such file or directory


In [ ]:
pip install chess


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
#nur nötig wenn man tpu brauchen will

pip install torch-xla
import torch_xla
import torch_xla.core.xla_model as xm

In [ ]:
def rook(strng):
  zaehler = 0
  for i in range(64):
    if strng[i] == '1':
      zaehler += 1
  return zaehler * 5

def bishop_knight(strng):
  zaehler = 0
  for i in range(128):
    if strng[i] == '1':
      zaehler += 1
  return 3 * zaehler

def queen(strng):
  zaehler = 0
  for i in range(64):
    if strng[i] == '1':
      zaehler += 1
  return 9 * zaehler

def pawn(strng):
  zaehler = 0
  for i in range(64):
    if strng[i] == '1':
      zaehler += 1
  return zaehler

Modell:

In [ ]:
import sqlite3

conn = sqlite3.connect('2021-07-31-lichess-evaluations-37MM.db')
cur = conn.cursor()

res = cur.execute('SELECT COUNT(id) FROM Evaluations')
print(res.fetchall())

[(37164639,)]


In [ ]:
def int_to_binary_64(num):
    binary_string = bin(num)[2:]  # Convert to binary and remove the '0b' prefix
    binary_string = binary_string.zfill(64)  # Zero-fill to ensure a length of 64 bits
    return binary_string

def int_to_binary_44(num):
    binary_string = bin(num)[2:]  # Convert to binary and remove the '0b' prefix
    binary_string = binary_string.zfill(4)  # Zero-fill to ensure a length of 64 bits
    return binary_string

def int_to_binary_6(num):
    binary_string = bin(num)[2:]  # Convert to binary and remove the '0b' prefix
    binary_string = binary_string.zfill(6)  # Zero-fill to ensure a length of 64 bits
    return binary_string

def int_to_binary_10(num):
    binary_string = bin(num)[2:]  # Convert to binary and remove the '0b' prefix
    binary_string = binary_string.zfill(10)  # Zero-fill to ensure a length of 64 bits
    return binary_string

def en_passant(board):
    if board.ep_square == None:
        return int_to_binary_10(65)
    return int_to_binary_10(int(board.ep_square))

In [ ]:
from peewee import *
import base64
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import chess

#db = SqliteDatabase('/content/gdrive/My Drive/shess_dataset.db')
db = SqliteDatabase('2021-07-31-lichess-evaluations-37MM.db')

class Evaluations(Model):
  id = IntegerField()
  fen = TextField()
  binary = BlobField()
  eval = FloatField()

  class Meta:
    database = db

  def binary_base64(self):
    return base64.b64encode(self.binary)
db.connect()

#LABEL_COUNT = len(dataset_0)
LABEL_COUNT = 37164639
print(LABEL_COUNT)


import os
import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, IterableDataset, random_split
from random import randrange
from tqdm import tqdm
import time


class EvaluationDataset(IterableDataset):
  def __init__(self, count):
    self.count = count
    self.idx = -1
  def __iter__(self):
    return self
  def __next__(self):
    self.idx += 1
    if self.idx == self.count:
      self.idx = 0
      return self[self.idx]
    return self[self.idx]
  def __len__(self):
    return self.count
  def __getitem__(self, idx):
    eval = Evaluations.get(Evaluations.id == idx+1)
    fen = eval.fen
    board = chess.Board(fen)
    bin_string=int_to_binary_64(int(board.pieces(chess.PAWN, chess.WHITE)))+int_to_binary_64(int(board.pieces(chess.ROOK, chess.WHITE)))+int_to_binary_64(int(board.pieces(chess.KNIGHT, chess.WHITE)))+int_to_binary_64(int(board.pieces(chess.BISHOP, chess.WHITE)))+int_to_binary_64(int(board.pieces(chess.QUEEN, chess.WHITE)))+int_to_binary_64(int(board.pieces(chess.KING, chess.WHITE)))+int_to_binary_64(int(board.pieces(chess.PAWN, chess.BLACK)))+int_to_binary_64(int(board.pieces(chess.ROOK, chess.BLACK)))+int_to_binary_64(int(board.pieces(chess.KNIGHT, chess.BLACK)))+int_to_binary_64(int(board.pieces(chess.BISHOP, chess.BLACK)))+int_to_binary_64(int(board.pieces(chess.QUEEN, chess.BLACK)))+int_to_binary_64(int(board.pieces(chess.KING, chess.BLACK)))+str(int(board.turn))
    #bin_string = bin_string[:769]
    points_white = pawn(bin_string[:64]) + rook(bin_string[64:128]) + bishop_knight(bin_string[128:256]) + queen(bin_string[256:320])
    points_black = pawn(bin_string[384:448]) + rook(bin_string[448:512]) + bishop_knight(bin_string[512:640]) + queen(bin_string[640:704])
    points = torch.tensor([points_white - points_black])
    bins = torch.tensor([float(digit) for digit in bin_string])
    bin = torch.cat((bins, points))
    ev = np.array([eval.eval]).astype(np.single)
    return {'binary':bin, 'eval':ev, 'id':idx+1}




class NeuralNetwork(nn.Module):
  def __init__(self, input_size, layer_sizes):
    super(NeuralNetwork, self).__init__()

    layers = []
    for i in range(len(layer_sizes) - 1):
      layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
      layers.append(nn.ReLU())

    # Remove the last ReLU to avoid an activation after the final layer
    layers.pop()

    self.seq = nn.Sequential(*layers)

  def forward(self, x):
    return self.seq(x)


def train_model(model, train_loader, device, num_epochs, learning_rate, batch):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    start_time = time.time()
    ziit = time.time()

    for epoch in range(num_epochs):
        running_loss = 0
        total = 0
        times = time.time()
        num_batches = len(train_loader)

        for batch_idx, data in enumerate(tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch')):
          if time.time() - ziit >= 1800:
            torch.save(model.state_dict(), 'ShessGPT.pth')
          if batch_idx >= num_batches:
            break

          bin = data["binary"].to(device)
          eval = data["eval"].to(device)
          optimizer.zero_grad()

          outputs = model(bin)

          loss = criterion(outputs, eval)

          loss.backward()
          optimizer.step()

          running_loss += loss.item()
          total += eval.size(0)

        print(f'Epoch {epoch + 1} complete - loss: {running_loss / (total / batch):.3f}')
        print(time.time() - start_time)

37164639


Hyperparameter setzen:

In [ ]:
# Hyperparameters
input_size = 770  # binary features are 800 bits
output_size = 1  # Single output for evaluation score
layer_sizes = [input_size,512, 256, 128, 64, 32, 16, output_size] # do not change first an last layer sizes. add as many layers inbetween as needed
learning_rate = 0.0001
num_epochs = 10
batch = 1024

In [ ]:
Auswahl device für trainingj

In [ ]:
#GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#TPU
#device = xm.xla_device()
#46
print(device)

cuda:0


KI Trainiern + speichern

In [ ]:
dataset_0 = EvaluationDataset(count=LABEL_COUNT)
trainloader = DataLoader(dataset_0, batch_size=batch)
model = NeuralNetwork(input_size, layer_sizes)
model.load_state_dict(torch.load('ShessGPT.pth'))
model.to(device)
train_model(model, trainloader, device, num_epochs, learning_rate, batch)

#save model

filename = f'/content/gdrive/My Drive/shess/chess_eval_model_cor_{len(layer_sizes)}_{learning_rate}_{num_epochs}_{batch}.pth'
torch.save(model.state_dict(),filename)

#save layer sizes

with open('/content/gdrive/My Drive/shess/layers.txt', 'a') as f:
    f.write("\n" + filename + ": " + ",".join(map(str, layer_sizes)))

Epoch 1/10: 100%|██████████| 36294/36294 [8:09:04<00:00,  1.24batch/s]


Epoch 1 complete - loss: 45.253
29344.896609783173


Epoch 2/10:  43%|████▎     | 15677/36294 [3:31:09<4:40:21,  1.23batch/s]

In [ ]:
torch.save(model.state_dict(), 'ShessGPT.pth')

In [ ]:
filename = f'/content/gdrive/My Drive/shess/chess_eval_model_cor_{len(layer_sizes)}_{learning_rate}_{num_epochs}_{batch}.pth'
torch.save(model.state_dict(),filename)

#save layer sizes

with open('/content/gdrive/My Drive/shess/layers.txt', 'a') as f:
    f.write("\n" + filename + ": " + ",".join(map(str, layer_sizes)))

Short test:

In [ ]:
filename = f'/content/gdrive/My Drive/shess/chess_eval_model_cor_{len(layer_sizes)}_{learning_rate}_{num_epochs}_{batch}.pth'

model = NeuralNetwork(input_size, layer_sizes)
for id in range(1,500):
  eval = dataset.get(dataset.id == id)
  model.load_state_dict(torch.load(filename))
  bin_string = ''.join(format(byte,'08b')for byte in eval.binary)
  bin = torch.tensor([float(digit) for digit in bin_string])
  print(eval.eval, model(bin))

0.29 tensor([-0.0750], grad_fn=<ViewBackward0>)
0.43 tensor([0.1149], grad_fn=<ViewBackward0>)
0.33 tensor([0.0763], grad_fn=<ViewBackward0>)
0.89 tensor([1.1437], grad_fn=<ViewBackward0>)
0.55 tensor([0.5740], grad_fn=<ViewBackward0>)
0.62 tensor([0.8629], grad_fn=<ViewBackward0>)
0.74 tensor([0.3793], grad_fn=<ViewBackward0>)
1.52 tensor([1.6069], grad_fn=<ViewBackward0>)
1.29 tensor([2.9819], grad_fn=<ViewBackward0>)
1.3 tensor([1.4329], grad_fn=<ViewBackward0>)
0.89 tensor([0.5052], grad_fn=<ViewBackward0>)
1.14 tensor([0.4887], grad_fn=<ViewBackward0>)
0.39 tensor([-0.1298], grad_fn=<ViewBackward0>)
0.5 tensor([0.8030], grad_fn=<ViewBackward0>)
-4.33 tensor([0.2886], grad_fn=<ViewBackward0>)
1.13 tensor([2.1386], grad_fn=<ViewBackward0>)
-4.77 tensor([1.3001], grad_fn=<ViewBackward0>)
-2.69 tensor([1.8958], grad_fn=<ViewBackward0>)
-2.7 tensor([1.3533], grad_fn=<ViewBackward0>)
-2.5 tensor([2.8225], grad_fn=<ViewBackward0>)
-2.64 tensor([1.5607], grad_fn=<ViewBackward0>)
-2.28 ten